<a href="https://colab.research.google.com/github/jorgedch/data-vis/blob/main/notebooks/Data-Vis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Challenge 1

# Data Visualization

In this notebook you'll find real-world tasks that will challenge your data visualization skills and your creativity. The idea is not only to make use of various visualization methods, but to experience the insights that can only be obtained through them and to use them as story-telling tools to communicate your findings.

Our tasks are divided in three levels of difficuty. The first one requires basic skills to explore a given dataset on a pre-defined datase, the second one asks you to implement a local solution in your machine and the third one requires to deploy the solution in a clound environment. These tasks also comprise three areas of application: <u>data-exploration</u>, <u>model-understanding</u> and <u>communication of results</u>. To be able to tackle them, they require that you understand the Machine Learning concepts, performance metrics and visualization tools explained in the following guide (Approx. time to read: 3 hours):

- https://pair-code.github.io/what-if-tool/learn/

## Beginner

To get started, let's take a look at an article published by the People+AI Research group of Google. In this article they talk about the _hidden bias_ contained in real-world datasets used to train Machine Learning models and how these models can hurt people.

- https://pair.withgoogle.com/explorables/hidden-bias

Can you think of another example where hidden bias in datasets can hurt people? Other times, hidden bias don't hurt people but simply make models perform worse in the production environment than in the training environment. What could be an example of this other situation?

_Your answers here..._

Next, we make use of the _What-If_ tool developed by the same group. This time you'll interact with multiple classification models to predict the yearly income of a person based on their census information. Explore the different features available and how each of them affects the predictions made by the models.

- https://pair-code.github.io/what-if-tool/demos/uci.html

Here are a couple of questions concerning _data-exploration_ and _model-understanding_:
- Take a look at the Features tab.
  - Are the numerical features uniformly distributed?
  - Which features are multimodal?
  - Are the categorical features balanced?
  - How would you ensure different types of features are not biased towards some values in your training dataset?
- Which features help the models to make fairer predictions?
- Which features have the most hidden bias that can hurt people?
- Which model makes the most accurate predictions?
- Which model has the best F1 score?
- Based on two features that you choose, which model makes fairer predictions? Which features did you choose?

_Your answers here..._

## Intermediate

Now let's take a look at the code behind the online dashboards and customize it to our needs. The following code is adapted from a [PAIR](https://colab.research.google.com/github/pair-code/what-if-tool/blob/master/WIT_Model_Comparison.ipynb) tutorial and allows you to change the model's hyper-parameters and the data used for training and visualization. 

Firstly, examine what are the hardware specs available for computation. To make sure that you have access to a GPU when running this notebook inside a Colab environment, go to _Edit ➞ Notebook settings ➞ Hardware accelerator_ and select _GPU_.

In [ ]:
! nvidia-smi

Install the What-If Tool widget if running in colab:

In [ ]:
try:
  import google.colab
  !pip install --upgrade witwidget
except:
  pass

Import all the required libraries:

In [ ]:
# Standard:
import functools

# Pip:
import numpy as np
import pandas as pd
import tensorflow as tf
from witwidget.notebook.visualization import WitConfigBuilder, WitWidget

Define helper functions:

In [ ]:
# Creates a tf feature spec from the dataframe and columns specified.
def create_feature_spec(df, columns=None):
    feature_spec = {}
    if columns == None:
        columns = df.columns.values.tolist()
    for f in columns:
        if df[f].dtype is np.dtype(np.int64):
            feature_spec[f] = tf.io.FixedLenFeature(shape=(), dtype=tf.int64)
        elif df[f].dtype is np.dtype(np.float64):
            feature_spec[f] = tf.io.FixedLenFeature(shape=(), dtype=tf.float32)
        else:
            feature_spec[f] = tf.io.FixedLenFeature(shape=(), dtype=tf.string)
    return feature_spec

# Creates simple numeric and categorical feature columns from a feature spec
# and a list of columns from that spec to use.
#
# NOTE: Models might perform better with some feature engineering such as bucketed
# numeric columns and hash-bucket/embedding columns for categorical features.
def create_feature_columns(columns, feature_spec):
    ret = []
    for col in columns:
        if feature_spec[col].dtype is tf.int64 or feature_spec[col].dtype is tf.float32:
            ret.append(tf.feature_column.numeric_column(col))
        else:
            ret.append(tf.feature_column.indicator_column(
                tf.feature_column.categorical_column_with_vocabulary_list(col, list(df[col].unique()))))
    return ret

# An input function for providing input to a model from tf.Examples
def tfexamples_input_fn(examples, feature_spec, label, mode=tf.estimator.ModeKeys.EVAL,
                       num_epochs=None, 
                       batch_size=64):
    def ex_generator():
        for i in range(len(examples)):
            yield examples[i].SerializeToString()
    dataset = tf.data.Dataset.from_generator(
      ex_generator, tf.dtypes.string, tf.TensorShape([]))
    if mode == tf.estimator.ModeKeys.TRAIN:
        dataset = dataset.shuffle(buffer_size=2 * batch_size + 1)
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(lambda tf_example: parse_tf_example(tf_example, label, feature_spec))
    dataset = dataset.repeat(num_epochs)
    return dataset

# Parses Tf.Example protos into features for the input function.
def parse_tf_example(example_proto, label, feature_spec):
    parsed_features = tf.io.parse_example(serialized=example_proto, features=feature_spec)
    target = parsed_features.pop(label)
    return parsed_features, target

# Converts a dataframe into a list of tf.Example protos.
def df_to_examples(df, columns=None):
    examples = []
    if columns == None:
        columns = df.columns.values.tolist()
    for index, row in df.iterrows():
        example = tf.train.Example()
        for col in columns:
            if df[col].dtype is np.dtype(np.int64):
                example.features.feature[col].int64_list.value.append(int(row[col]))
            elif df[col].dtype is np.dtype(np.float64):
                example.features.feature[col].float_list.value.append(row[col])
            elif row[col] == row[col]:
                example.features.feature[col].bytes_list.value.append(row[col].encode('utf-8'))
        examples.append(example)
    return examples

# Converts a dataframe column into a column of 0's and 1's based on the provided test.
# Used to force label columns to be numeric for binary classification using a TF estimator.
def make_label_column_numeric(df, label_column, test):
    df[label_column] = np.where(test(df[label_column]), 1, 0)

Read training dataset from CSV:

In [ ]:
# Set the path to the CSV containing the dataset to train on.
csv_path = 'https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data'

# Set the column names for the columns in the CSV. If the CSV's first line is a header line containing
# the column names, then set this to None.
csv_columns = [
  "Age", "Workclass", "fnlwgt", "Education", "Education-Num", "Marital-Status",
  "Occupation", "Relationship", "Race", "Sex", "Capital-Gain", "Capital-Loss",
  "Hours-per-week", "Country", "Over-50K"]

# Read the dataset from the provided CSV and print out information about it.
df = pd.read_csv(csv_path, names=csv_columns, skipinitialspace=True)

df

Specify input columns and column to predict:

In [ ]:
# Set the column in the dataset you wish for the model to predict
label_column = 'Over-50K'

# Make the label column numeric (0 and 1), for use in our model.
# In this case, examples with a target value of '>50K' are considered to be in
# the '1' (positive) class and all other examples are considered to be in the
# '0' (negative) class.
make_label_column_numeric(df, label_column, lambda val: val == '>50K')

# Set list of all columns from the dataset we will use for model input.
input_features = [
  'Age', 'Workclass', 'Education', 'Marital-Status', 'Occupation',
  'Relationship', 'Race', 'Sex', 'Capital-Gain', 'Capital-Loss',
  'Hours-per-week', 'Country']

# Create a list containing all input features and the label column
features_and_labels = input_features + [label_column]

Convert dataset to tf.Example protos:

In [ ]:
examples = df_to_examples(df)

Create a feature spec and input function for both classifiers:

In [ ]:
feature_spec = create_feature_spec(df, features_and_labels)
train_inpf = functools.partial(tfexamples_input_fn, examples, feature_spec, label_column)

Create and train the **linear** classifier:

In [ ]:
num_steps = 1000

# Define and train the classifier
classifier = tf.estimator.LinearClassifier(
    feature_columns=create_feature_columns(input_features, feature_spec)
)
classifier.train(train_inpf, steps=num_steps)

Create and train the **DNN** classifier:

In [ ]:
num_steps_2 = 1000

# Define and train the classifier
classifier2 = tf.estimator.DNNClassifier(
    feature_columns=create_feature_columns(input_features, feature_spec),
    hidden_units=[128, 64, 32]
)
classifier2.train(train_inpf, steps=num_steps_2)

Invoke What-If Tool for test data and the trained models:

In [ ]:
num_datapoints = 1000
tool_height_in_px = 1000

# Load up the test dataset
test_csv_path = 'https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test'
test_df = pd.read_csv(test_csv_path, names=csv_columns, skipinitialspace=True, skiprows=1)
make_label_column_numeric(test_df, label_column, lambda val: val == '>50K.')
test_examples = df_to_examples(test_df[0:num_datapoints])

# Setup the tool with the test examples and the trained classifier
config_builder = WitConfigBuilder(test_examples[0:num_datapoints])
config_builder = config_builder.set_estimator_and_feature_spec(classifier, feature_spec)
config_builder = config_builder.set_compare_estimator_and_feature_spec(classifier2, feature_spec)
config_builder = config_builder.set_label_vocab(['Under 50K', 'Over 50K'])

WitWidget(config_builder, height=tool_height_in_px)

Make the following modifications and answer the corresponding questions:

- Organize datapoints by setting X-axis scatter to "inference score 1" and Y-axis scatter to "inference score 2" to see how each datapoint differs in score between the linear model (1) and DNN model (2). Points off the diagonal have differences in results between the two models.
  - Are there patterns of which datapoints don't agree between the two models?
  - If you set the ground truth feature dropdown in the "Performance + Fairness" tab to "Over-50K", then you can color or bin the datapoints by "inference correct 1" or "inference correct 2". Are there patterns of which datapoints are incorrect for model 1? For model 2?

- Explore performance of the two models through the confusion matrices in the "Performance + Fairness" tab. Which model is best? Train either model for longer and see if you can change this. Are the rates of errors (false positives and false negatives) that the two models make different?
  - Click the "optimize threshold" button to set the optimal positive classification threshold for each model based on the current cost ratio of 1. How do those thresholds and the resulting confusion matrices differ?
    - Change the cost ratio and optimize the threshold again. How does the threshold and performance change on the two models?
  - Slice the dataset by features, such as "sex" or "race". Does either model have more-equal performance between slices?
    - Use the threshold optimization buttons to set optimal thresholds based on the different fairness constraints. How does performance between slices differ between the two models. Does one require larger differences in threshold values per slice to achieve the desired constraint?

- Looking at the create_feature_columns function in the "Define helper methods" cell, categorical features use one-hot encodings in the model. Perhaps change a many-valued categorical feature, such as education to use an embedding layer. Does anything change in the model behavior (can look through partial dependence plots as one way to investigate).

_Your answers here..._

## Advanced